In [3]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from collections import defaultdict
from scipy.stats import pearsonr

In [1]:
datasetTweetsCSVPath = "./dataset/tweets.csv"
datasetUsersCSVPath  = "./dataset/users.csv"

In [4]:
df_tweets = pd.read_csv(datasetTweetsCSVPath, sep=',', index_col=0)

In [18]:
df_tweets.head()

,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,text
id,,,,,,,,,
509354017856950272,327746321,0,0,0,0,0,0,2019-09-11 14:53:55,"If man is a little lower than angels, then ang..."
583002592529121280,333722906,1,0,0,0,0,1,2020-04-01 20:27:04,"""@BestWSHHVids: how do you say these words wit..."
461498835362013185,2379755827,0,0,0,0,0,1,2019-05-02 13:34:31,@LOLatComedy awsome
528808127366692864,466226882,0,0,0,0,0,0,2019-11-04 07:17:37,Stephen Hawkins: i buchi neri non esistono se ...
575336690904006656,1355537995,114,0,0,1,0,1,2020-03-11 16:45:31,RT @tibbs_montris: So ready for Wednesday!


In [16]:
df_tweets.dtypes

user_id           object
retweet_count     object
reply_count       object
favorite_count    object
num_hashtags      object
num_urls          object
num_mentions      object
created_at        object
text              object
dtype: object

In [5]:
df_tweets.index

Index(['509354017856950272', '583002592529121280', '461498835362013185',
       '528808127366692864', '575336690904006656', '583448653542596609',
        '79540398988595200', '344253939950182400', '335131564512006144',
       '424575175074385920',
       ...
       '499812877029900288', '427109081338621952',               'idjn',
       '459163209170046976', '329829994169786368', '588535254207467520',
       '580420653666402304',    '718157017970961', '441301348676415488',
       '486479812614451200'],
      dtype='object', name='id', length=13664696)

In [6]:
df_tweets.shape

(13664696, 9)

In [24]:
df_tweets.isnull().any()

user_id            True
retweet_count      True
reply_count        True
favorite_count     True
num_hashtags       True
num_urls           True
num_mentions       True
created_at        False
text               True
dtype: bool

In [70]:
print("Distinct Values in user_id: \t\t", df_tweets.user_id.unique())
print("Distinct Values in retweet_count:\t", df_tweets.retweet_count.unique())
print("Distinct Values in reply_count: \t", df_tweets.reply_count.unique())
print("Distinct Values in favorite_count: \t", df_tweets.favorite_count.unique())
print("Distinct Values in num_hashtags: \t", df_tweets.num_hashtags.unique())
print("Distinct Values in num_urls: \t\t", df_tweets.num_urls.unique())
print("Distinct Values in num_mentions: \t", df_tweets.num_mentions.unique())
print("Distinct Values in created_at: \t", df_tweets.created_at.unique())
print("Distinct Values in text: \t", df_tweets.text.unique())

Distinct Values in user_id: 		 ['327746321' '333722906' '2379755827' ... 'e5uobk0249' 'fig6' 'tzn9s6bwj']
Distinct Values in retweet_count:	 ['0' '1' '114' ... 'mpc2' '8d5hy52d' 'rf24duucpb']
Distinct Values in reply_count: 	 ['0' '0.0' nan ... 'krr4jjgnz' 'z8w0qvba' 'otsduzr']
Distinct Values in favorite_count: 	 ['0' '2' '1' ... 'a58k' '09vtlcms9k' 'l25suv5']
Distinct Values in num_hashtags: 	 ['0' '1' '2' ... '6xppy0' 'lwpe' 'kmmt7p21sf']
Distinct Values in num_urls: 		 ['0' '2' '1' ... '0cyoe1vpd' 'y2o' 'ix0n']
Distinct Values in num_mentions: 	 ['0' '1' '3' ... 'd8j' 'mstagcw' '8155oi']
Distinct Values in created_at: 	 ['2019-09-11 14:53:55' '2020-04-01 20:27:04' '2019-05-02 13:34:31' ...
 '2019-04-06 10:51:15' '2019-01-27 16:01:54' '2019-07-10 12:00:00']
Distinct Values in text: 	 ['If man is a little lower than angels, then angels should reform.'
 '"@BestWSHHVids: how do you say these words with a straight face?!\nhttp://t.co/RWGMf3jwWy"\n\nPurchasing'
 '@LOLatComedy awsome' ...

In [ ]:
distinct_number_of_tweets = len(df_tweets.text.unique())
total_number_of_tweets    = len(df_tweets.text)
if (distinct_number_of_tweets == total_number_of_tweets):
    print("There aren't duplicate tweets")
else:
    print("There are duplicate tweets, maybe some missing tweets")

There are duplicate tweets, maybe some missing tweets


In [25]:
df_tweets.describe()

,user_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,text
count,13447413,13227562,13016818,13017154,12607172,13016073,12810531,13664696,13126975
unique,222285,229167,158619,158763,104779,157145,131538,8127084,6778478
top,497404180,0,0,0,0,0,0,2020-04-04 03:43:02,"Ci sedemmo dalla parte del torto, visto che tu..."
freq,4600,9419784,11790440,9467209,10470209,9894198,7561627,133,6932


In [26]:
df_users = pd.read_csv(datasetUsersCSVPath, sep=',', index_col=0)

In [28]:
df_users.dtypes

name               object
lang               object
bot                 int64
created_at         object
statuses_count    float64
dtype: object

In [27]:
df_users.describe()

,bot,statuses_count
count,11508.000000,11109.000000
mean,0.531456,5883.312359
std,0.499031,19073.007305
min,0.000000,0.000000
25%,0.000000,41.000000
50%,1.000000,68.000000
75%,1.000000,2916.000000
max,1.000000,399555.000000


In [30]:
df_users['bot'].unique()

array([1, 0])

In [56]:
df_users["statuses_count"].apply(float.is_integer).all()

False

In [81]:
#joined_df=pd.merge(df_tweets, df_users, left_on='user_id', right_on='id')
#joined_df